In [1]:
import pandas as pd
import numpy as np

In [3]:
sp500 = pd.read_csv("SP500/sp500_1.csv")
# Rename columns
sp500.columns = ['Date', 'sp500', 'sp500+1','ret_sp500', 'ret_sp500+1']
sp500 = sp500.dropna()
sLength = len(sp500['Date'])
# add the bullMarket column with random ints (to be replaced in next cell)
sp500['bullMarket'] = pd.Series(np.random.randn(sLength), index=sp500.index)

# Create bullMarkert for ret_sp500+1
bull = []
for data in sp500['ret_sp500+1']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
sp500['bullMarket'] = bull
# print(sp500)

sp500.shape
# print(sp500)

(14275, 6)

In [27]:
tdata = pd.read_csv("Total_Data_no_dots.csv",low_memory=False)

result = pd.merge(sp500, tdata, on='Date', how='left')

result  = result.dropna()

# assume we want to predict the raw index of sp500 at T+1, given all the info at T

y_reg = result['ret_sp500+1']

y_cal = result['bullMarket']
 
# list(result)

result = result.drop(columns=['VOX Close', 'DGS1', 'DGS10', 'T10Y2Y', 'DBAA', 'USD1MTD156N', 'DCOILBRENTEU', 'DTWEXB', 'DEXUSEU', 'T10YIE', 'T5YIFR', 'BAMLH0A0HYM2EY'])

x = result.drop(columns=['Date','sp500_x','sp500+1','ret_sp500+1','Unnamed: 0'])

# print(result.dtypes)

In [29]:
# Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import KFold

kf = KFold(x.shape[0], n_folds=2, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y_reg.iloc[train_index], y_reg.iloc[test_index]
    y_train_cal, y_test_cal = y_cal.iloc[train_index], y_cal.iloc[test_index]

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)

    y_pred = regr.predict(x_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))
    
print( "the main take way: the mean squared error is always within 1% ")


MSE:  0.8108592714839336
MSE:  0.8220161493992437
the main take way: the mean squared error is always within 1% 


In [31]:
# clssification 

# decision tree CART algorithm

from sklearn import tree
from sklearn.metrics import confusion_matrix
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train_cal)
predicted = clf.predict(x_test)
# print (predicted)
c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: 'perfect' prediction, overfitting problem")


[[2132    0]
 [   0 2242]]
precision:  1.0
recall:  1.0
the main take way: 'perfect' prediction, overfitting problem


In [26]:
# naive Bayes 
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(x_train, y_train_cal).predict(x_test)

predicted= gnb.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is slightly better than a coin flip")

[[ 120  143]
 [2012 2099]]
precision:  0.9362176628010704
recall:  0.5105813670639747


In [34]:
# Support Vector Classification

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x_train, y_train_cal) 

predicted= clf.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is way more better than naive Bayes ")

[[1063    0]
 [1069 2242]]
precision:  1.0
recall:  0.6771368166717004
the main take way: High Precision, recall is way more better than naive Bayes 
